In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Activation, Input, Flatten
from keras.models import Model, Sequential, load_model, Input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.applications.inception_v3 import preprocess_input

In [ ]:
#Read the csv file
df = pd.read_csv('data/labels.csv')

#Sort by frequency and extract top 10
dist = df.groupby('breed').count().rename(columns={'id':'freq'})
most_common = dist.sort_values(by='freq',ascending=False)
top_10 = [i for i in most_common[:10].index]

In [ ]:
top_10_freq = most_common.loc[top_10,:].sort_values(by='freq',ascending=False)
baseline = top_10_freq['freq'][0]/(top_10_freq['freq'].sum())
print('baseline',baseline)

print('class imbalance', top_10_freq['freq'][0], top_10_freq['freq'][9])
print(top_10_freq['freq'][0]/top_10_freq['freq'][9])

In [ ]:
df = df[df['breed'].isin(top_10)]
X = df['id']
y = df['breed']
X_train_id, X_test_id, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_id, X_val_id, y_train, y_val = train_test_split(X_train_id, y_train, test_size=0.1)

In [ ]:
X_train_id.reset_index(inplace=True, drop=True)
X_val_id.reset_index(inplace=True, drop=True)
X_test_id.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_val.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

In [ ]:
folders = ['train10', 'test10','val10']
for folder in folders: 
    for breed in top_10:
        if not os.path.exists('data/{}/{}'.format(folder,breed)):
            os.makedirs('data/{}/{}'.format(folder, breed))

In [ ]:
for i in tqdm(range(len(X_train_id))):
    image_path = 'data/train/{}.jpg'.format(X_train_id[i])
    img = Image.open(image_path)
    img.save("data/train10/{}/{}.jpg".format(y_train[i], X_train_id[i]))

In [ ]:
for i in tqdm(range(len(X_val_id))):
    image_path = 'data/train/{}.jpg'.format(X_val_id[i])
    img = Image.open(image_path)
    img.save("data/val10/{}/{}.jpg".format(y_val[i], X_val_id[i]))

In [ ]:
for i in tqdm(range(len(X_test_id))):
    image_path = 'data/train/{}.jpg'.format(X_test_id[i])
    img = Image.open(image_path)
    img.save("data/test10/{}/{}.jpg".format(y_test[i], X_test_id[i]))

In [ ]:
dim = 150

In [ ]:
datagen = ImageDataGenerator(
          rotation_range=20,
          rescale=1./255,
          zoom_range=0.2,
          shear_range=0.2,
          horizontal_flip=True,
          )

image_path = 'data/train/{}.jpg'.format(X_train_id[10])
img = load_img(image_path,target_size=(dim, dim))
x = img_to_array(img) 
x = x.reshape((1,) + x.shape)

i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='data/preview',
                          save_prefix='sample',
                          save_format='jpg'):
    i += 1
    if i >= 5:
        break  # otherwise the generator would loop indefinitely

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3), input_shape=(dim,dim,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(optimizer=adam,loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rotation_range=20,
        rescale=1./255,
        zoom_range=0.2,
        shear_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'data/train10',
        target_size=(dim, dim),
        batch_size=batch_size,
        class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
        'data/val10',
        target_size=(dim, dim),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        'data/test10',
        target_size=(dim, dim),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
h = model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=val_generator,
        validation_steps=800 // batch_size)

In [ ]:
model.evaluate_generator(test_generator, steps=50)

In [ ]:
print('minimum val_loss is {} at epoch {}'.format(min(h.history['val_loss']), np.argmin(h.history['val_loss'])+1))
print('maximum accuracy is {} at epoch {}'.format(max(h.history['val_acc']), np.argmax(h.history['val_acc'])+1))

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(h.history['loss'])
plt.plot(h.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')

plt.subplot(1, 2, 2)
plt.plot(h.history['acc'])
plt.plot(h.history['val_acc'])
plt.legend(['acc', 'val_acc'])
plt.ylabel('acc')
plt.xlabel('epoch')